In [1]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib

In [16]:
gb00 = joblib.load('gb00.sav')
gb01 = joblib.load('gb01.sav')
gb02 = joblib.load('gb02.sav')
gb03 = joblib.load('gb03.sav')
gb04 = joblib.load('gb04.sav')
mlp00 = joblib.load('mlp00.sav')
mlp01 = joblib.load('mlp01.sav')
mlp02 = joblib.load('mlp02.sav')
mlp03 = joblib.load('mlp03.sav')
mlp04 = joblib.load('mlp04.sav')

In [4]:
mod.predict_proba(X_test)

NameError: name 'X_test' is not defined

In [14]:
# Read in data
mod_data = pd.read_csv('appointments_clean.csv')

# Split into X and y
X = mod_data.drop(
    ['PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay', 
     'Neighbourhood', 'No-show', 'NoShow'], 
    axis = 1
)
y = mod_data.NoShow

# Test/train split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 314
)

# Scale data
scaler = MinMaxScaler().fit(X_train)

X_train_mm = scaler.transform(X_train)
X_test_mm  = scaler.transform(X_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [6]:
mod.predict_proba(X_test)

array([[0.94524543, 0.05475457],
       [0.93989396, 0.06010604],
       [0.94263482, 0.05736518],
       ...,
       [0.34025407, 0.65974593],
       [0.93648136, 0.06351864],
       [0.4032586 , 0.5967414 ]])

In [7]:
def prob_summary(clf, data, actual):
    
    # Generate predicted probabilities
    probs = clf.predict_proba(data)
    probs = pd.Series([i[1] for i in probs])
    
    # Add probabilities to actuals in dataframe
    probs_df = pd.concat([probs, actual.reset_index()], axis = 1).reset_index()
    
    # Clean dataframe
    probs_df.columns = ['level_0', 'Prob', 'index', 'Actual']
    probs_df         = probs_df.drop(['level_0', 'index'], axis = 1)
    
    # Round probabilities to nearest tenth
    probs_df['RoundedProb'] = (probs_df['Prob']).round(1)
    
    # Summarize
    summary = probs_df.groupby(
        'RoundedProb'
    ).agg(
        {
            'Actual':['count', 'sum']
        }
    )
    
    # Fix column names
    summary.columns = ["_".join(i) for i in summary.columns.ravel()]
    
    # Calculate predicted percentages
    summary['Actual_percentage'] = (summary['Actual_sum'] / summary['Actual_count']).round(2)
    return(summary)

In [17]:
print('Gradient Boosting')
print(prob_summary(gb00, X_test, y_test), '\n')

print('Gradient Boosting')
print(prob_summary(gb03, X_test_mm, y_test), '\n')

print('Gradient Boosting')
print(prob_summary(gb04, X_test_mm, y_test), '\n')

print('Multilayer Perceptron')
print(prob_summary(mlp00, X_test, y_test), '\n')

print('Multilayer Perceptron')
print(prob_summary(mlp03, X_test_mm, y_test), '\n')

print('Multilayer Perceptron')
print(prob_summary(mlp04, X_test_mm, y_test), '\n')

Gradient Boosting
             Actual_count  Actual_sum  Actual_percentage
RoundedProb                                             
0.0                 23091           0               0.00
0.1                     1           0               0.00
0.2                   988         211               0.21
0.3                   391         108               0.28
0.4                   352         144               0.41
0.5                    66          38               0.58
0.6                  1378         869               0.63
0.7                  2519        1803               0.72
0.8                  3010        2420               0.80
0.9                  1360        1186               0.87
1.0                     3           3               1.00 

Gradient Boosting
             Actual_count  Actual_sum  Actual_percentage
RoundedProb                                             
0.0                 22581        3835               0.17
0.1                 10559        2931             